In [ ]:
import os
from coffea import hist
from coffea import util
import numpy as np
import itertools

# Start by loading the unweighted coffea outputs containing the efficiency numerator and denominators

In [ ]:
dir = 'CoffeaOutputs/UnweightedOutputs/'

    # ---- With MC Weights ---- #

# QCD_unweighted = util.load(dir+'TTbarResCoffea_QCD_unweighted_output_dask_8-01-21_MC_efficiency_NoAnacats_100chunks_10000chunksize.coffea')
# TTbar_unweighted = util.load(dir+'TTbarResCoffea_TTbar_unweighted_output_dask_8-01-21_MC_efficiency_NoAnacats_100chunks_10000chunksize.coffea')

    # ---- With MC Weights ---- #  Strange TTbar results from typo in code

# QCD_unweighted = util.load(dir+'TTbarResCoffea_QCD_unweighted_output_futures_9-08-21_MC_efficiency_NoAnacats_DeltaR_test_100chunks_10000chunksize.coffea')
# TTbar_unweighted = util.load(dir+'TTbarResCoffea_TTbar_unweighted_output_futures_9-08-21_MC_efficiency_NoAnacats_DeltaR_test_100chunks_10000chunksize.coffea')

    # ---- No MC Weights ---- # 

# QCD_unweighted = util.load(dir+'TTbarResCoffea_QCD_unweighted_output_dask_9-09-21_MC_efficiency_NoAnacats_NoWeights_100chunks_10000chunksize.coffea')
# TTbar_unweighted = util.load(dir+'TTbarResCoffea_TTbar_unweighted_output_dask_9-09-21_MC_efficiency_NoAnacats_NoWeights_100chunks_10000chunksize.coffea')

    # ---- Fixed Typos ---- #

# QCD_unweighted = util.load(dir+'TTbarResCoffea_QCD_unweighted_output_futures_9-10-21_MC_efficiency_NoAnacats_DeltaR_test_100chunks_10000chunksize.coffea')
# TTbar_unweighted = util.load(dir+'TTbarResCoffea_TTbar_unweighted_output_futures_9-10-21_MC_efficiency_NoAnacats_DeltaR_test_100chunks_10000chunksize.coffea')

    # ---- Fixed Typos + Full Datasets ---- #

QCD_unweighted = util.load(dir+'TTbarResCoffea_QCD_unweighted_output_futures_9-11-21_MC_efficiency_NoAnacats_FullDataset.coffea')
TTbar_unweighted = util.load(dir+'TTbarResCoffea_TTbar_unweighted_output_futures_9-11-21_MC_efficiency_NoAnacats_FullDataset.coffea')



In [ ]:
Datasets = {
    'QCD': QCD_unweighted,
    'TTbar': TTbar_unweighted
}

# The cutflow can be checked if desired 

In [ ]:
for dataset,output in Datasets.items():
    print("-------" + dataset + " Cutflow--------")
    for i,j in output['cutflow'].items():        
        print( '%20s : %12d' % (i,j) )

# Define a function that can make directories to store png files of the histograms (if desired)

In [ ]:
def mkdir_p(mypath):
    '''Creates a directory. equivalent to using mkdir -p on the command line'''

    from errno import EEXIST
    from os import makedirs,path

    try:
        makedirs(mypath)
    except OSError as exc: # Python >2.5
        if exc.errno == EEXIST and path.isdir(mypath):
            pass
        else: raise

In [ ]:
def DoesDirectoryExist(mypath): #extra precaution (Probably overkill...)
    '''Checks to see if Directory exists before running mkdir_p'''
    import os.path
    from os import path
    
    if path.exists(mypath):
        pass
    else:
        mkdir_p(mypath)

# Define where you want the main directory (place to store the directories with the new png plots)

In [ ]:
maindirectory = os.getcwd() 

Various plotting options I've copied and pasted over from the other analysis notebooks.  Could be of use ;)

In [ ]:
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import warnings
import re # regular expressions
warnings.filterwarnings("ignore")

stack_tag1_opts = {'marker': '.', 'markersize': 10., 'color': 'b', 'elinewidth': 1}
stack_tag2_opts = {'marker': 's', 'markersize': 5., 'color': 'g', 'elinewidth': 1}
legend_tag_opts = {'labels':['', '', 'DeepTag', 'DeepTag MD']}
stack_ttbar_opts = {'alpha': 0.8, 'edgecolor':(0,0,0,0.3), 'color': 'red'}
stack_background_opts = {'alpha': 0.8, 'edgecolor':(0,0,0,0.3), 'color': 'yellow'}
stack_error_opts = {'label':'Stat. Unc.', 'hatch':'///', 'facecolor':'None', 'edgecolor':(0,0,0,.5), 'linewidth': 0}
data_err_opts = {'linestyle': 'none', 'marker': '.', 'markersize': 10., 'color': 'k', 'elinewidth': 1}

# Plot (and save) the flavor tagging efficiencies as functions of $p_T$ 

In [ ]:
SaveDirectory = maindirectory + '/FlavorTagEfficiencies_NoAnacat/images/1D/flavortags_pt/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
for dataset,output in Datasets.items(): 
    plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 18,
    'axes.labelsize': 18,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12
    })
    fig, (ax01, ax02, ax11, ax12) = plt.subplots(1, 4, figsize=(17,5), sharex=True)

    b_eff_numerator_pt_s01 = output['b_eff_numerator_s01'].integrate('subjeteta').integrate('dataset', dataset)
    b_eff_denominator_pt_s01 = output['b_eff_denominator_s01'].integrate('subjeteta').integrate('dataset', dataset)
    c_eff_numerator_pt_s01 = output['c_eff_numerator_s01'].integrate('subjeteta').integrate('dataset', dataset)
    c_eff_denominator_pt_s01 = output['c_eff_denominator_s01'].integrate('subjeteta').integrate('dataset', dataset)
    udsg_eff_numerator_pt_s01 = output['udsg_eff_numerator_s01'].integrate('subjeteta').integrate('dataset', dataset)
    udsg_eff_denominator_pt_s01 = output['udsg_eff_denominator_s01'].integrate('subjeteta').integrate('dataset', dataset)
    
    b_eff_pt_s01 = hist.plotratio(num = b_eff_numerator_pt_s01, denom = b_eff_denominator_pt_s01, 
                                  ax=ax01, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                                  unc = 'num')
    c_eff_pt_s01 = hist.plotratio(num = c_eff_numerator_pt_s01, denom = c_eff_denominator_pt_s01, 
                                  ax=ax01, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                                  unc = 'num')
    udsg_eff_pt_s01 = hist.plotratio(num = udsg_eff_numerator_pt_s01, denom = udsg_eff_denominator_pt_s01, 
                                  ax=ax01, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                  unc = 'num')
    
    ax01.set_title(r'SubJet$_{01}\left(p_T\right)$')
    ax01.set_ylabel('b-Tag Efficiency')
    ax01.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    b_eff_numerator_pt_s02 = output['b_eff_numerator_s02'].integrate('subjeteta').integrate('dataset', dataset)
    b_eff_denominator_pt_s02 = output['b_eff_denominator_s02'].integrate('subjeteta').integrate('dataset', dataset)
    c_eff_numerator_pt_s02 = output['c_eff_numerator_s02'].integrate('subjeteta').integrate('dataset', dataset)
    c_eff_denominator_pt_s02 = output['c_eff_denominator_s02'].integrate('subjeteta').integrate('dataset', dataset)
    udsg_eff_numerator_pt_s02 = output['udsg_eff_numerator_s02'].integrate('subjeteta').integrate('dataset', dataset)
    udsg_eff_denominator_pt_s02 = output['udsg_eff_denominator_s02'].integrate('subjeteta').integrate('dataset', dataset)
    
    b_eff_pt_s02 = hist.plotratio(num = b_eff_numerator_pt_s02, denom = b_eff_denominator_pt_s02, 
                                  ax=ax02, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                                  unc = 'num')
    c_eff_pt_s02 = hist.plotratio(num = c_eff_numerator_pt_s02, denom = c_eff_denominator_pt_s02, 
                                  ax=ax02, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                                  unc = 'num')
    udsg_eff_pt_s02 = hist.plotratio(num = udsg_eff_numerator_pt_s02, denom = udsg_eff_denominator_pt_s02, 
                                  ax=ax02, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                  unc = 'num')
    
    ax02.set_title(r'SubJet$_{02}\left(p_T\right)$')
    ax02.set_ylabel('')
    ax02.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    b_eff_numerator_pt_s11 = output['b_eff_numerator_s11'].integrate('subjeteta').integrate('dataset', dataset)
    b_eff_denominator_pt_s11 = output['b_eff_denominator_s11'].integrate('subjeteta').integrate('dataset', dataset)
    c_eff_numerator_pt_s11 = output['c_eff_numerator_s11'].integrate('subjeteta').integrate('dataset', dataset)
    c_eff_denominator_pt_s11 = output['c_eff_denominator_s11'].integrate('subjeteta').integrate('dataset', dataset)
    udsg_eff_numerator_pt_s11 = output['udsg_eff_numerator_s11'].integrate('subjeteta').integrate('dataset', dataset)
    udsg_eff_denominator_pt_s11 = output['udsg_eff_denominator_s11'].integrate('subjeteta').integrate('dataset', dataset)
    
    b_eff_pt_s11 = hist.plotratio(num = b_eff_numerator_pt_s11, denom = b_eff_denominator_pt_s11, 
                                  ax=ax11, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                                  unc = 'num')
    c_eff_pt_s11 = hist.plotratio(num = c_eff_numerator_pt_s11, denom = c_eff_denominator_pt_s11, 
                                  ax=ax11, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                                  unc = 'num')
    udsg_eff_pt_s11 = hist.plotratio(num = udsg_eff_numerator_pt_s11, denom = udsg_eff_denominator_pt_s11, 
                                  ax=ax11, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                  unc = 'num')
    
    ax11.set_title(r'SubJet$_{11}\left(p_T\right)$')
    ax11.set_ylabel('')
    ax11.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    b_eff_numerator_pt_s12 = output['b_eff_numerator_s12'].integrate('subjeteta').integrate('dataset', dataset)
    b_eff_denominator_pt_s12 = output['b_eff_denominator_s12'].integrate('subjeteta').integrate('dataset', dataset)
    c_eff_numerator_pt_s12 = output['c_eff_numerator_s12'].integrate('subjeteta').integrate('dataset', dataset)
    c_eff_denominator_pt_s12 = output['c_eff_denominator_s12'].integrate('subjeteta').integrate('dataset', dataset)
    udsg_eff_numerator_pt_s12 = output['udsg_eff_numerator_s12'].integrate('subjeteta').integrate('dataset', dataset)
    udsg_eff_denominator_pt_s12 = output['udsg_eff_denominator_s12'].integrate('subjeteta').integrate('dataset', dataset)
    
    b_eff_pt_s12 = hist.plotratio(num = b_eff_numerator_pt_s12, denom = b_eff_denominator_pt_s12, 
                                  ax=ax12, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                                  unc = 'num')
    c_eff_pt_s12 = hist.plotratio(num = c_eff_numerator_pt_s12, denom = c_eff_denominator_pt_s12, 
                                  ax=ax12, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                                  unc = 'num')
    udsg_eff_pt_s12 = hist.plotratio(num = udsg_eff_numerator_pt_s12, denom = udsg_eff_denominator_pt_s12, 
                                  ax=ax12, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                  unc = 'num')
    
    ax12.set_title(r'SubJet$_{12}\left(p_T\right)$')
    ax12.set_ylabel('')
    ax12.set_ylim(bottom=0., top=1.)
    leg = ax11.legend(labels=[r'$b_{eff}$',r'$c_{eff}$',r'$udsg_{eff}$'], bbox_to_anchor=(0.7, 1.4), loc='upper right')
    fig.suptitle(dataset + ' Sample')

    #filename = dataset + '_flavor_efficiency_pt_largerbins_' + '.png'
#     filename = dataset + '_flavor_efficiency_pt' + '.png'
#     plt.savefig(SaveDirectory+filename, bbox_inches="tight")

# Plot (and save) the flavor tagging efficiencies as functions of $\eta$ 

In [ ]:
SaveDirectory = maindirectory + '/FlavorTagEfficiencies_NoAnacat/images/1D/flavortags_eta/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
for dataset,output in Datasets.items(): 
    plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 18,
    'axes.labelsize': 18,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12
    })
    fig, (ax01, ax02, ax11, ax12) = plt.subplots(1, 4, figsize=(17,5), sharex=True)

    b_eff_numerator_pt_s01 = output['b_eff_numerator_s01'].integrate('subjetpt').integrate('dataset', dataset)
    b_eff_denominator_pt_s01 = output['b_eff_denominator_s01'].integrate('subjetpt').integrate('dataset', dataset)
    c_eff_numerator_pt_s01 = output['c_eff_numerator_s01'].integrate('subjetpt').integrate('dataset', dataset)
    c_eff_denominator_pt_s01 = output['c_eff_denominator_s01'].integrate('subjetpt').integrate('dataset', dataset)
    udsg_eff_numerator_pt_s01 = output['udsg_eff_numerator_s01'].integrate('subjetpt').integrate('dataset', dataset)
    udsg_eff_denominator_pt_s01 = output['udsg_eff_denominator_s01'].integrate('subjetpt').integrate('dataset', dataset)
    
    b_eff_pt_s01 = hist.plotratio(num = b_eff_numerator_pt_s01, denom = b_eff_denominator_pt_s01, 
                                  ax=ax01, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                                  unc = 'num')
    c_eff_pt_s01 = hist.plotratio(num = c_eff_numerator_pt_s01, denom = c_eff_denominator_pt_s01, 
                                  ax=ax01, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                                  unc = 'num')
    udsg_eff_pt_s01 = hist.plotratio(num = udsg_eff_numerator_pt_s01, denom = udsg_eff_denominator_pt_s01, 
                                  ax=ax01, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                  unc = 'num')
    
    ax01.set_title(r'SubJet$_{01}\left(\eta\right)$')
    ax01.set_ylabel('b-Tag Efficiency')
    ax01.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    b_eff_numerator_pt_s02 = output['b_eff_numerator_s02'].integrate('subjetpt').integrate('dataset', dataset)
    b_eff_denominator_pt_s02 = output['b_eff_denominator_s02'].integrate('subjetpt').integrate('dataset', dataset)
    c_eff_numerator_pt_s02 = output['c_eff_numerator_s02'].integrate('subjetpt').integrate('dataset', dataset)
    c_eff_denominator_pt_s02 = output['c_eff_denominator_s02'].integrate('subjetpt').integrate('dataset', dataset)
    udsg_eff_numerator_pt_s02 = output['udsg_eff_numerator_s02'].integrate('subjetpt').integrate('dataset', dataset)
    udsg_eff_denominator_pt_s02 = output['udsg_eff_denominator_s02'].integrate('subjetpt').integrate('dataset', dataset)
    
    b_eff_pt_s02 = hist.plotratio(num = b_eff_numerator_pt_s02, denom = b_eff_denominator_pt_s02, 
                                  ax=ax02, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                                  unc = 'num')
    c_eff_pt_s02 = hist.plotratio(num = c_eff_numerator_pt_s02, denom = c_eff_denominator_pt_s02, 
                                  ax=ax02, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                                  unc = 'num')
    udsg_eff_pt_s02 = hist.plotratio(num = udsg_eff_numerator_pt_s02, denom = udsg_eff_denominator_pt_s02, 
                                  ax=ax02, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                  unc = 'num')
    
    ax02.set_title(r'SubJet$_{02}\left(\eta\right)$')
    ax02.set_ylabel('')
    ax02.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    b_eff_numerator_pt_s11 = output['b_eff_numerator_s11_largerbins'].integrate('subjetpt').integrate('dataset', dataset)
    b_eff_denominator_pt_s11 = output['b_eff_denominator_s11_largerbins'].integrate('subjetpt').integrate('dataset', dataset)
    c_eff_numerator_pt_s11 = output['c_eff_numerator_s11_largerbins'].integrate('subjetpt').integrate('dataset', dataset)
    c_eff_denominator_pt_s11 = output['c_eff_denominator_s11_largerbins'].integrate('subjetpt').integrate('dataset', dataset)
    udsg_eff_numerator_pt_s11 = output['udsg_eff_numerator_s11_largerbins'].integrate('subjetpt').integrate('dataset', dataset)
    udsg_eff_denominator_pt_s11 = output['udsg_eff_denominator_s11_largerbins'].integrate('subjetpt').integrate('dataset', dataset)
    
    b_eff_pt_s11 = hist.plotratio(num = b_eff_numerator_pt_s11, denom = b_eff_denominator_pt_s11, 
                                  ax=ax11, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                                  unc = 'num')
    c_eff_pt_s11 = hist.plotratio(num = c_eff_numerator_pt_s11, denom = c_eff_denominator_pt_s11, 
                                  ax=ax11, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                                  unc = 'num')
    udsg_eff_pt_s11 = hist.plotratio(num = udsg_eff_numerator_pt_s11, denom = udsg_eff_denominator_pt_s11, 
                                  ax=ax11, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                  unc = 'num')
    
    ax11.set_title(r'SubJet$_{11}\left(\eta\right)$')
    ax11.set_ylabel('')
    ax11.set_ylim(bottom=0., top=1.)

    # ------------------------------------------------------------------------------ #

    b_eff_numerator_pt_s12 = output['b_eff_numerator_s12'].integrate('subjetpt').integrate('dataset', dataset)
    b_eff_denominator_pt_s12 = output['b_eff_denominator_s12'].integrate('subjetpt').integrate('dataset', dataset)
    c_eff_numerator_pt_s12 = output['c_eff_numerator_s12'].integrate('subjetpt').integrate('dataset', dataset)
    c_eff_denominator_pt_s12 = output['c_eff_denominator_s12'].integrate('subjetpt').integrate('dataset', dataset)
    udsg_eff_numerator_pt_s12 = output['udsg_eff_numerator_s12'].integrate('subjetpt').integrate('dataset', dataset)
    udsg_eff_denominator_pt_s12 = output['udsg_eff_denominator_s12'].integrate('subjetpt').integrate('dataset', dataset)
    
    b_eff_pt_s12 = hist.plotratio(num = b_eff_numerator_pt_s12, denom = b_eff_denominator_pt_s12, 
                                  ax=ax12, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'b', 'elinewidth': 1},
                                  unc = 'num')
    c_eff_pt_s12 = hist.plotratio(num = c_eff_numerator_pt_s12, denom = c_eff_denominator_pt_s12, 
                                  ax=ax12, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'g', 'elinewidth': 1},
                                  unc = 'num')
    udsg_eff_pt_s12 = hist.plotratio(num = udsg_eff_numerator_pt_s12, denom = udsg_eff_denominator_pt_s12, 
                                  ax=ax12, clear=False,
                                  error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                  unc = 'num')
    
    ax12.set_title(r'SubJet$_{12}\left(\eta\right)$')
    ax12.set_ylabel('')
    ax12.set_ylim(bottom=0., top=1.)
    leg = ax11.legend(labels=[r'$b_{eff}$',r'$c_{eff}$',r'$udsg_{eff}$'], bbox_to_anchor=(0.7, 1.4), loc='upper right')
    fig.suptitle(dataset + ' Sample')

#     filename = dataset + '_flavor_efficiency_eta' + '.png'
#     plt.savefig(SaveDirectory+filename, bbox_inches="tight")

# Define function to plot a 2D ratio histogram given two coffea.hist objects (the numerator and denominator). #
# Now, we will be able to see the efficiencies as functions of both $p_T$ and $\eta$.

In [ ]:
import mplhep as hep
from hist.intervals import ratio_uncertainty

In [ ]:
def plotratio2d(numerator, denominator, ax=None, cmap='Blues', cbar=True):
    NumeratorAxes = numerator.axes()
    DenominatorAxes = denominator.axes()
    
    # integer number of bins in this axis #
    NumeratorAxis1_BinNumber = NumeratorAxes[0].size - 3 # Subtract 3 to remove overflow
    NumeratorAxis2_BinNumber = NumeratorAxes[1].size - 3
    
    DenominatorAxis1_BinNumber = DenominatorAxes[0].size - 3 
    DenominatorAxis2_BinNumber = DenominatorAxes[1].size - 3 
    
    if(NumeratorAxis1_BinNumber != DenominatorAxis1_BinNumber 
       or NumeratorAxis2_BinNumber != DenominatorAxis2_BinNumber):
        raise Exception('Numerator and Denominator axes are different sizes; Cannot perform division.')
    else:
        Numerator = numerator.to_hist()
        Denominator = denominator.to_hist()

        ratio = Numerator / Denominator.values()

#         err_up, err_down = ratio_uncertainty(Numerator.values(), Denominator_fixed, 'poisson-ratio')

#         labels = []
#         for ra, u, d in zip(ratio.values().ravel(), err_up.ravel(), err_down.ravel()):
#             ra, u, d = f'{ra:.2f}', f'{u:.2f}', f'{d:.2f}'
#             st = '$'+ra+'_{-'+d+'}^{+'+u+'}$'
#             labels.append(st)
#         labels = np.array(labels).reshape(NumeratorAxis1_BinNumber,NumeratorAxis2_BinNumber)
        
        return hep.hist2dplot(ratio, ax=ax, cmap=cmap, norm=colors.Normalize(0.,1.), cbar=cbar)

# Plot b-tag $\epsilon\left(p_T,\ \eta\right)$
# Can either plot with small bins $(50\times50)$ or large bins $(10\times10)$

In [ ]:
SaveDirectory = maindirectory + '/FlavorTagEfficiencies_NoAnacat/images/2D/btags/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
for dataset,output in Datasets.items():
    plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 18,
    'axes.labelsize': 18,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12
    })
    fig, (ax01, ax02, ax11, ax12) = plt.subplots(1, 4, figsize=(15,5), sharex=True)

    b_eff_numerator_s01 = output['b_eff_numerator_s01'].integrate('dataset', dataset)
    b_eff_denominator_s01 = output['b_eff_denominator_s01'].integrate('dataset', dataset)

    b_eff_s01 = plotratio2d(b_eff_numerator_s01, b_eff_denominator_s01, ax=ax01, cbar=False)

    ax01.set_title(r'b-tag $\epsilon_{01}\left(p_T,\ \eta\right)$')

    #----------------------------------------------------------------------------------------#

    b_eff_numerator_s02 = output['b_eff_numerator_s02'].integrate('dataset', dataset)
    b_eff_denominator_s02 = output['b_eff_denominator_s02'].integrate('dataset', dataset)

    b_eff_s02 = plotratio2d(b_eff_numerator_s02, b_eff_denominator_s02, ax=ax02, cbar=False)

    ax02.set_title(r'b-tag $\epsilon_{02}\left(p_T,\ \eta\right)$')
    ax02.set_ylabel('')
    ax02.set_yticks([])

    #----------------------------------------------------------------------------------------#

    b_eff_numerator_s11 = output['b_eff_numerator_s11'].integrate('dataset', dataset)
    b_eff_denominator_s11 = output['b_eff_denominator_s11'].integrate('dataset', dataset)

    b_eff_s11 = plotratio2d(b_eff_numerator_s11, b_eff_denominator_s11, ax=ax11, cbar=False)

    ax11.set_title(r'b-tag $\epsilon_{11}\left(p_T,\ \eta\right)$')
    ax11.set_ylabel('')
    ax11.set_yticks([])

    #----------------------------------------------------------------------------------------#

    b_eff_numerator_s12 = output['b_eff_numerator_s12'].integrate('dataset', dataset)
    b_eff_denominator_s12 = output['b_eff_denominator_s12'].integrate('dataset', dataset)

    b_eff_s12 = plotratio2d(b_eff_numerator_s12, b_eff_denominator_s12, ax=ax12)#, cmap='plasma')

    ax12.set_title(r'b-tag $\epsilon_{12}\left(p_T,\ \eta\right)$')
    ax12.set_ylabel('')
    ax12.set_yticks([])

    #----------------------------------------------------------------------------------------#
    fig.suptitle(dataset + ' Sample; Category')

#         filename = dataset + '_btag_efficiency_' + str(icat) + '.png'
#         plt.savefig(SaveDirectory+filename, bbox_inches="tight")

In [ ]:
SaveDirectory = maindirectory + '/FlavorTagEfficiencies_NoAnacat/images/2D/btags/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
for dataset,output in Datasets.items():
    plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 18,
    'axes.labelsize': 18,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12
    })
    fig, (ax01, ax02, ax11, ax12) = plt.subplots(1, 4, figsize=(15,5), sharex=True)

    b_eff_numerator_s01 = output['b_eff_numerator_s01_largerbins'].integrate('dataset', dataset)
    b_eff_denominator_s01 = output['b_eff_denominator_s01_largerbins'].integrate('dataset', dataset)

    b_eff_s01 = plotratio2d(b_eff_numerator_s01, b_eff_denominator_s01, ax=ax01, cbar=False)

    ax01.set_title(r'b-tag $\epsilon_{01}\left(p_T,\ \eta\right)$')

    #----------------------------------------------------------------------------------------#

    b_eff_numerator_s02 = output['b_eff_numerator_s02_largerbins'].integrate('dataset', dataset)
    b_eff_denominator_s02 = output['b_eff_denominator_s02_largerbins'].integrate('dataset', dataset)

    b_eff_s02 = plotratio2d(b_eff_numerator_s02, b_eff_denominator_s02, ax=ax02, cbar=False)

    ax02.set_title(r'b-tag $\epsilon_{02}\left(p_T,\ \eta\right)$')
    ax02.set_ylabel('')
    ax02.set_yticks([])

    #----------------------------------------------------------------------------------------#

    b_eff_numerator_s11 = output['b_eff_numerator_s11_largerbins'].integrate('dataset', dataset)
    b_eff_denominator_s11 = output['b_eff_denominator_s11_largerbins'].integrate('dataset', dataset)

    b_eff_s11 = plotratio2d(b_eff_numerator_s11, b_eff_denominator_s11, ax=ax11, cbar=False)

    ax11.set_title(r'b-tag $\epsilon_{11}\left(p_T,\ \eta\right)$')
    ax11.set_ylabel('')
    ax11.set_yticks([])

    #----------------------------------------------------------------------------------------#

    b_eff_numerator_s12 = output['b_eff_numerator_s12_largerbins'].integrate('dataset', dataset)
    b_eff_denominator_s12 = output['b_eff_denominator_s12_largerbins'].integrate('dataset', dataset)

    b_eff_s12 = plotratio2d(b_eff_numerator_s12, b_eff_denominator_s12, ax=ax12)#, cmap='plasma')

    ax12.set_title(r'b-tag $\epsilon_{12}\left(p_T,\ \eta\right)$')
    ax12.set_ylabel('')
    ax12.set_yticks([])

    #----------------------------------------------------------------------------------------#
    fig.suptitle(dataset + ' Sample')

#         filename = dataset + '_btag_efficiency_large_bins' + str(icat) + '.png'
#         plt.savefig(SaveDirectory+filename, bbox_inches="tight")